# Evaluate and Use Models from Model Garden

## Task 2. Prepare prompts for model evaluation

In [1]:
!pip install --upgrade --quiet google-cloud-aiplatform \
    google-cloud-logging openai 'anthropic[vertex]' httpx

### Setup and Initialize Vertex AI 

Run the following cells to install and import packages and initialize Vertex AI.

In [1]:
import google.cloud.logging
from google.cloud import aiplatform
from IPython.display import Markdown
import logging
import pandas as pd
import random

import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig
from vertexai.evaluation import (
    MetricPromptTemplateExamples,
    EvalTask,
    PairwiseMetric,
    PointwiseMetric,
)

# Sets Pandas to display content in wider columns
pd.set_option("display.max_colwidth", None)

# Do not remove logging section
client = google.cloud.logging.Client()
client.setup_logging()

# TODO Add your project and location to initialize vertxai
PROJECT_ID, = !gcloud config get project
LOCATION = "us-central1"

vertexai.init(project=PROJECT_ID, location=LOCATION)

pd.set_option("display.max_colwidth", None)

### Define Prompts for Evaluation

In [15]:
prompt_template = """
    You are a technical architect.

    Prepare a high-level project plan for how a solution to the
    following PROJECT REQUEST could be developed using Google Cloud.
    
    We don't want to begin projects that we can't deliver well in
    under four months, so also weigh in on whether you think this
    is a good time for us to attempt this project.
    
    PROJECT REQUEST:
    {project_request}
"""

In [16]:
project_requests = [
    """
    Inventory Management with Real-time Visibility.
    Retail division needs to track inventory across all physical stores
    and e-commerce channels with real-time accuracy. Key pain point
    is stockouts and overstock leading to lost sales and increased
    holding costs. They're looking for a cloud solution that integrates
    seamlessly with existing POS systems and offers predictive analytics
    for demand forecasting.
    """,
    """
    AI-Driven Material Selection & Performance Prediction.
    Product Development is interested in AI's ability to analyze material properties
    and predict their performance under various conditions within the
    context of their industrial designs. Wants assistance in identifying
    the most suitable and cost-effective materials early in the design process.
    """,
    """
    Optimize Yield & Reduce Input Costs.
    Agriculture department specifically mentioned a desire to increase crop
    output (e.g., by X%) while simultaneously decreasing expenses on water,
    fertilizer, and pesticides (e.g., by Y%) through more precise application
    and predictive insights.
    """,
    """
    Enhanced Predictive Maintenance for Fleet Operations.
    Distribution department expressed a strong need to leverage AI/ML on their
    cloud platform for proactive identification of potential component failures
    across their delivery fleet. Key drivers are reducing unscheduled downtime,
    optimizing maintenance schedules, and minimizing operational disruption.
    Requires ingestion of vast amounts of sensor data (IoT), flight logs, and
    maintenance records.
    """,
    """
    Inaccurate Vehicle Valuation.
    Distribution department struggles with inconsistent and often delayed real-time
    pricing for used vehicles, leading to lost profit margins and slower inventory
    turnover. Needs a solution that leverages AI to provide highly accurate,
    dynamic valuation based on a wide range of data points (market trends,
    condition, history, local demand).
    """

]

In [17]:
# TODO
# Update the code so that the variable `prompts` contains a list of
# strings, applying the `prompt_template` to each item in `project_requests`
prompts=[]
for request in project_requests:
    # Format the template with the current request
    full_prompt = prompt_template.format(project_request=request)
    prompts.append(full_prompt)

# print(prompts[0])

In [18]:
# Check your work

assert len(prompts) == 5, "You don't have a response for each project_request."
assert type(prompts[0]) == str, ("Make sure your prompts are strings.")

## Task 3. Get Llama Responses using the OpenAI SDK

In [21]:
from google.auth import default, transport
import openai

MAX_TOKENS = 4096

In [22]:
credentials, _ = default()
auth_request = transport.requests.Request()
credentials.refresh(auth_request)

In [23]:
LLAMA_LOCATION = "us-east5"
MAAS_ENDPOINT = f"{LLAMA_LOCATION}-aiplatform.googleapis.com"
LLAMA_MODEL_ID = "meta/llama-4-scout-17b-16e-instruct-maas"

# TODO
base_url = (
    f"https://{MAAS_ENDPOINT}/v1/projects/{PROJECT_ID}/"
    f"locations/{LLAMA_LOCATION}/endpoints/openapi"
)
# Implement an openai_cient to access the Llama 4 model
# using the endpoint and model ID defined above

openai_client = openai.OpenAI(
    base_url=base_url,
    api_key=credentials.token,
)

In [25]:
llama_responses = []

# TODO
# Update the code below to populate the list llama_responses
# with just the text string of Llama 4's response to each prompt

llama_responses = []

## ✍️ Update the code below to populate the list llama_responses
## with just the text string of Llama 4's response to each prompt

### TODO: Start Implementation ###
for prompt in prompts:
    # 1. Prepare the messages list for the Chat Completion API
    messages = [
        {"role": "user", "content": prompt}
    ]

    # 2. Call the Llama model via the OpenAI client
    try:
        completion = openai_client.chat.completions.create(
            model=LLAMA_MODEL_ID,
            messages=messages,
            max_tokens=MAX_TOKENS,
        )

        # 3. Extract the text string from the response and append to the list
        # The text content is found at: completion.choices[0].message.content
        response_text = completion.choices[0].message.content
        llama_responses.append(response_text)
        
    except Exception as e:
        print(f"An error occurred while processing prompt '{prompt}': {e}")
        llama_responses.append(f"ERROR: {e}")

In [27]:
# Check your work

assert len(prompts) == len(llama_responses), "You don't have a response for each prompt."
assert type(llama_responses[0]) == str, ("Extract just the text response, "
                                          "not the entire response object.")
# View one response
# Markdown(llama_responses[0])

## Task 4. Get Claude Responses using the Anthropic SDK

In [29]:
from anthropic import AnthropicVertex

In [30]:
CLAUDE_MODEL = "claude-sonnet-4@20250514"
CLAUDE_LOCATIONS = ["us-east5", "europe-west4", "GLOBAL"]
LOCATION= CLAUDE_LOCATIONS[0]
# TODO
# Implement an anthropic_client to access the Claude Sonnet 4 model
# using the model ID defined above


anthropic_client = AnthropicVertex(
    project_id=PROJECT_ID,
    region=LOCATION
)

In [31]:
claude_responses = []

# TODO
# Update the code below to populate the list claude_responses
# with just the text string of Clade Sonnet 4's response to each prompt

for prompt in prompts:
    try:
        # 1. Prepare the messages list
        messages = [
            {"role": "user", "content": prompt}
        ]

        # 2. Call the Claude model via the AnthropicVertex client
        message_response = anthropic_client.messages.create(
            model=CLAUDE_MODEL,
            max_tokens=256,
            messages=messages,
        )

        # 3. Extract the text content
        # The content is a list of TextBlock objects. We access the first block
        # and then the 'text' attribute.
        response_text = message_response.content[0].text
        claude_responses.append(response_text)
        
    except Exception as e:
        print(f"An error occurred while processing prompt '{prompt}': {e}")
        claude_responses.append(f"ERROR: {e}")

In [34]:
# Check your work

assert len(prompts) == len(claude_responses), "You don't have a response for each prompt."
assert type(claude_responses[0]) == str, ("Extract just the text response, "
                                          "not the entire response object.")

# View one response
# Markdown(claude_responses[0])

## Task 5. Conduct a pairwise evaluation

### Explore Metrics and Select Evaluation Criteria

In [ ]:
MetricPromptTemplateExamples.list_example_metric_names()

In [ ]:
print(MetricPromptTemplateExamples.Pairwise.TEXT_QUALITY.metric_prompt_template)

### Prepare and Execute the Evaluation Task

In [33]:
# TODO
# Provide all the variables required for the metric
# selected above

eval_dataset = pd.DataFrame({
    'prompt': prompts,
    'baseline_model_response': llama_responses,
    'response': claude_responses
})

In [35]:
selected_metric = MetricPromptTemplateExamples.Pairwise.TEXT_QUALITY

# TODO
# Complete the definition of the EvalTask using the dataset defined
# above and the selected metric. Keep the provided experiment name.

pairwise_eval_task =  EvalTask(
    dataset=eval_dataset,
    metrics=[selected_metric]
)

In [36]:
pairwise_result = pairwise_eval_task.evaluate()

Computing metrics with a total of 5 Vertex Gen AI Evaluation Service API requests.


100%|██████████| 5/5 [00:25<00:00,  5.06s/it]

All 5 metric requests are successfully computed.
Evaluation Took:25.364229331999923 seconds


In [37]:
# TODO
# Display the result's summary_metrics
pairwise_result.summary_metrics

{'row_count': 5,
 'pairwise_text_quality/candidate_model_win_rate': 0.0,
 'pairwise_text_quality/baseline_model_win_rate': 1.0}

In [38]:
# TODO
# Display the result's metrics_table table
pairwise_result.metrics_table

prompt  \
0                                               \n    You are a technical architect.\n\n    Prepare a high-level project plan for how a solution to the\n    following PROJECT REQUEST could be developed using Google Cloud.\n    \n    We don't want to begin projects that we can't deliver well in\n    under four months, so also weigh in on whether you think this\n    is a good time for us to attempt this project.\n    \n    PROJECT REQUEST:\n    \n    Inventory Management with Real-time Visibility.\n    Retail division needs to track inventory across all physical stores\n    and e-commerce channels with real-time accuracy. Key pain point\n    is stockouts and overstock leading to lost sales and increased\n    holding costs. They're looking for a cloud solution that integrates\n    seamlessly with existing POS systems and offers predictive analytics\n    for demand forecasting.\n    \n   
1                                                                                                                  \n    You are a technical architect.\n\n    Prepare a high-level project plan for how a solution to the\n    following PROJECT REQUEST could be developed using Google Cloud.\n    \n    We don't want to begin projects that we can't deliver well in\n    under four months, so also weigh in on whether you think this\n    is a good time for us to attempt this project.\n    \n    PROJECT REQUEST:\n    \n    AI-Driven Material Selection & Performance Prediction.\n    Product Development is interested in AI's ability to analyze material properties\n    and predict their performance under various conditions within the\n    context of their industrial designs. Wants assistance in identifying\n    the most suitable and cost-effective materials early in the design process.\n    \n   
2                                                                                                                                                                                     \n    You are a technical architect.\n\n    Prepare a high-level project plan for how a solution to the\n    following PROJECT REQUEST could be developed using Google Cloud.\n    \n    We don't want to begin projects that we can't deliver well in\n    under four months, so also weigh in on whether you think this\n    is a good time for us to attempt this project.\n    \n    PROJECT REQUEST:\n    \n    Optimize Yield & Reduce Input Costs.\n    Agriculture department specifically mentioned a desire to increase crop\n    output (e.g., by X%) while simultaneously decreasing expenses on water,\n    fertilizer, and pesticides (e.g., by Y%) through more precise application\n    and predictive insights.\n    \n   
3  \n    You are a technical architect.\n\n    Prepare a high-level project plan for how a solution to the\n    following PROJECT REQUEST could be developed using Google Cloud.\n    \n    We don't want to begin projects that we can't deliver well in\n    under four months, so also weigh in on whether you think this\n    is a good time for us to attempt this project.\n    \n    PROJECT REQUEST:\n    \n    Enhanced Predictive Maintenance for Fleet Operations.\n    Distribution department expressed a strong need to leverage AI/ML on their\n    cloud platform for proactive identification of potential component failures\n    across their delivery fleet. Key drivers are reducing unscheduled downtime,\n    optimizing maintenance schedules, and minimizing operational disruption.\n    Requires ingestion of vast amounts of sensor data (IoT), flight logs, and\n    maintenance records.\n    \n   
4                                                                                        \n    You are a technical architect.\n\n    Prepare a high-level project plan for how a solution to the\n    following PROJECT REQUEST could be developed using Google Cloud.\n    \n    We don't want to begin projects that we can't deliver well in\n    under four months, so also weigh in on whether you think 

In [39]:
# Explore the first result's explanation
print(pairwise_result.metrics_table.loc[0,'pairwise_text_quality/explanation'])

BASELINE response provides a complete high-level project plan, including a detailed timeline and a thorough feasibility and risk assessment, directly addressing all parts of the prompt. CANDIDATE response is incomplete, ending abruptly at Week 10 of a projected 16-week timeline, and thus fails to fully address the instruction to create a plan for a project under four months or provide a comprehensive assessment.


## Test the CodeGemma deployment

In [41]:
PROJECT_NUMBER= 837112514786

In [ ]:
import google.cloud.logging
import logging

from google.cloud import aiplatform

# Do not remove logging section
client = google.cloud.logging.Client()
client.setup_logging()


# Replace your project number and endpoint id
ENDPOINT_ID= ''

endpoint_resource_name="projects/PROJECT_NUMBER/locations/us-central1/endpoints/ENDPOINT_ID"
endpoint=aiplatform.Endpoint(endpoint_resource_name)


In [ ]:

prompt = "Write a function to list n Fibonacci numbers in Python." 
max_tokens = 500 
temperature = 1.0 
top_p = 1.0  
top_k = 1 
raw_response = True

instances = [
    {
        "prompt": prompt,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "top_p": top_p,
        "top_k": top_k,
        "raw_response": raw_response
    },
]

response = endpoint.predict(
    instances=instances
)

# "<|file_separator|>" is the end of the file token.
for prediction in response.predictions:
    print(prediction.split("<|file_separator|>")[0])


print(response.deployed_model_id)

# Do not remove logging section
log_message = f"Fibonacci function: {response}"
logging.info(log_message)

## Perform Cleanup

In [ ]:
# Delete all endpoints and their deployed models
endpoints = aiplatform.Endpoint.list()

if not endpoints:
    print("No endpoints found in this project/region.")
else:
    for endpoint in endpoints:
        print(f"\nEndpoint: {endpoint.display_name} ({endpoint.resource_name})")

        deployed_models = endpoint.list_models()
        for dm in deployed_models:
            print(f"  Undeploying model ID: {dm.id}")            
#             TODO Uncomment the statement below to undeploy a model
            endpoint.undeploy(deployed_model_id=dm.id, sync=True)
            print(f"  Undeployed model ID: {dm.id}")

        print("  Deleting endpoint...")
        endpoint.delete(sync=True)
        print(f"  Deleted endpoint: {endpoint.display_name}")

print("\nDeleting models...")
models = aiplatform.Model.list()

for model in models:
    print(f"  Deleting model: {model.display_name}")
    # TODO Uncomment the statement below to delete the model.
    model.delete(sync=True)
    print("  Deleted.")
    
# Do not remove logging section
log_message = "Deleted endpoints"
logging.info(log_message)
